<h1>Data Cleaning</h1>

In this notebook we clean the provided datasets by usding natural language processing methods on them.

In [ ]:
# Needed imports
import nltk, pickle, re
import pandas as pd

# need to use once to download nltk (natural language processing library) on your computer.
# nltk.download()

<h3>Tweet extraction</h3>

This function extracts tweet data from a given filename and puts it in a dataframe. We then apply it to labeled tweets datasets (to get positives and negatives) and the test dataset.

In [ ]:
def get_tweets(filename):
    
    # Read the data file
    with open("twitter-datasets/" + filename, "r", encoding="utf8") as myfile:
        data = myfile.readlines()
        
    # Make a dataframe out of the data
    return pd.DataFrame(data)

In [ ]:
negative_DF = get_tweets("train_neg_full.txt")
positive_DF = get_tweets("train_pos_full.txt")
test_DF = get_tweets("test_data.txt")

In [ ]:
negative_DF.head(10)

<h3>Tweet Formatting</h3>

In this function, we format the data to suit our needs. As such, we put everything to lowercase, remove unwanted elements such as usertags, urls, retweet mentions (rt) and lone characters. We also replace non-alphabetical characters by spaces and sequences of 3 times or more of the same character by a single occurence of said character.

We also remove the (English) stop words found in the the nltk package (words such as "I", "you", etc...). Finally, the strings are replaced by lists of words. This function is then applied to each dataset.

In [ ]:
def format_tweets(tweets):
    
    # These are stop words that we want to take out from the tweets
    lang_set = nltk.corpus.stopwords.words('english')
    
    # Put everything in lowercase
    tweets[0] = tweets[0].astype(str).str.lower()
    
    # The replacement instructions for below, which:
    # - remove usertags
    # - remove urls
    # - remove retweets ("rt")
    # - replace anything that is not letters by a space
    # - remove lone characters
    # - replace sequences of 3 times the same letter or more by a single occurence of the character
    replacements = [
        ("<user>", ''),
        ("<url>", ''),
        (r'\brt\b', ''),
        (r'[^a-z]+', ' '),
        (r'\b\w\b', ''),
        (r'([a-zA-Z])\1{2,}', r'\1')
    ]
    
    # Apply the replacements instructions
    for key, value in replacements:
        tweets[0] = tweets[0].str.replace(key, value)
            
    # Tokenize each tweet
    tweets[0] = tweets[0].str.split()
    
    # Remove the stop words
    tweets[0] = tweets[0].apply(lambda tweet: [word for word in tweet if word not in lang_set])
        
    return tweets

In [ ]:
negative_DF = format_tweets(negative_DF)
positive_DF = format_tweets(positive_DF)
test_DF = format_tweets(test_DF)

In [ ]:
negative_DF.head(10)

<h3>Stemming and Lemmatizing</h3>

Now that our tweets are cleaned, we apply stemming (crude chopping of the end of the words) and lemming (process that actually looks into a vocabulary to try to get the roots of words) to each word in the list to make words such as "love" and "lover" to be recognized as the same in order to increase efficiency down the road. 

In [ ]:
def stem_and_lem(tweets, stemmer, lemmer):
    tweets['stemmed'] = tweets[0].apply(lambda tweet: [stemmer.stem(word) for word in tweet])
    tweets['lemmed'] = tweets[0].apply(lambda tweet: [lemmatizer.lemmatize(word) for word in tweet])
    tweets['both'] = tweets['lemmed'].apply(lambda tweet: [stemmer.stem(word) for word in tweet])
    
    return tweets

In [ ]:
# Generating the stemmer and lemmatizer
stemmer = nltk.stem.snowball.SnowballStemmer('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
negative_DF = stem_and_lem(negative_DF, stemmer, lemmatizer)
positive_DF = stem_and_lem(positive_DF, stemmer, lemmatizer)
test_DF = stem_and_lem(test_DF, stemmer, lemmatizer)

In [ ]:
negative_DF.head(10)

<h3>Save data</h3>

All we have to do now is save our results in txt files so that they can be used in our different methods. We also save the DataFrames in pickle files to use them instead for efficiency.

In [ ]:
def save_tweets(tweets, filename):
    
    # Put the stemmed and lemmetized tweets back to string form
    data = tweets['lemmed'].apply(lambda x: ' '.join(x))
    
    # Save to file
    with open("twitter-datasets/" + filename, "w", encoding="utf8") as myfile:
        data.to_csv(myfile, index=False)

In [ ]:
save_tweets(negative_DF, "train_neg_proc.txt")
save_tweets(positive_DF, "train_pos_proc.txt")
save_tweets(test_DF, "test_data_proc.txt")

In [ ]:
with open('outputs/train_neg_proc.pkl', 'wb') as f:
    pickle.dump(negative_DF, f, pickle.HIGHEST_PROTOCOL)
    
with open('outputs/train_pos_proc.pkl', 'wb') as f:
    pickle.dump(positive_DF, f, pickle.HIGHEST_PROTOCOL)
    
with open('outputs/test_data_proc.pkl', 'wb') as f:
    pickle.dump(test_DF, f, pickle.HIGHEST_PROTOCOL)